In [1]:
######## Image Object Detection Using Tensorflow-trained Classifier #########
#
# Author: Evan Juras
# Date: 1/15/18
# Description: 
# This program uses a TensorFlow-trained classifier to perform object detection.
# It loads the classifier uses it to perform object detection on an image.
# It draws boxes and scores around the objects of interest in the image.

## Some of the code is copied from Google's example at
## https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb

## and some is copied from Dat Tran's example at
## https://github.com/datitran/object_detector_app/blob/master/object_detection_app.py

## but I changed it to make it more understandable to me.

# Import packages
import os
import cv2
import numpy as np
import tensorflow as tf
import sys
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch.autograd import Variable
import PIL
from os import listdir

# Import utilites
from utils import label_map_util
from utils import visualization_utils as vis_util

# Name of the directory containing the object detection module we're using
MODEL_NAME = 'inference_graph_fgo'
#file_dir = 'D:/object_detection/tf_model_dir/models/research/object_detection/images_fgo/test/'
file_dir = 'F:/fgo_multi_label/test/'

save_dir = 'D:/object_detection/tf_model_dir/models/research/object_detection/images_fgo_face/detected/'
crop_path = 'D:/object_detection/tf_model_dir/models/research/object_detection/images_fgo_face/crop_holder/'
#IMAGE_LOC = 'images_dice/train/'



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")


# Grab path to current working directory
CWD_PATH = os.getcwd()

# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,'frozen_inference_graph.pb')

# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH,'training_fgo','labelmap.pbtxt')

# Path to image
#PATH_TO_IMAGE = os.path.join(CWD_PATH,IMAGE_NAME)

# Number of classes the object detector can identify
NUM_CLASSES = 1

# Load the label map.
# Label maps map indices to category names, so that when our convolution
# network predicts `5`, we know that this corresponds to `king`.
# Here we use internal utility functions, but anything that returns a
# dictionary mapping integers to appropriate string labels would be fine
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)

# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')


file_list = []
for i in listdir(file_dir):
    if 'xml' not in i[-4:]:
        file_list.append(i)

target_dir = 'F:/fgo_multi_label/test/'
file_list = []
for i in listdir(target_dir):
    if 'xml' not in i[-4:]:
        file_list.append(i)        

files_done = 0
#annoy_index = AnnoyIndex(len(file_list))
list_of_heads = []
list_of_images = []
save_dir_list = []
for i in file_list:

    #if files_done % 50 == 0:
    #    print(files_done)
    #IMAGE_NAME = file_dir+i
    PATH_TO_IMAGE = file_dir+i
    #print(PATH_TO_IMAGE)
    # Load image using OpenCV and
    # expand image dimensions to have shape: [1, None, None, 3]
    # i.e. a single-column array, where each item in the column has the pixel RGB value
    image = cv2.imread(PATH_TO_IMAGE)
    #image = cv2.resize(image,(1920,1080))

    image_expanded = np.expand_dims(image, axis=0)
    try:
        image_for_cropping = image.copy()
    except:
        print('error with this image {}'.format(i))
        continue
    try:
        # Perform the actual detection by running the model with the image as input
        (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: image_expanded})
    except:
        print('no idea?...')
        continue
    #print(boxes,scores,classes,num)
    # Draw the results of the detection (aka 'visulaize the results')
    #print(np.squeeze(classes).astype(np.int32))
    
    image, class_list,box_list = vis_util.visualize_boxes_and_labels_on_image_array(
            image,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=8,
            min_score_thresh=0.80)

    for idx in range(len(box_list)):
        ymin = int(box_list[idx][0])
        xmin = int(box_list[idx][1])
        ymax = int(box_list[idx][2])
        xmax = int(box_list[idx][3])


        crop_img = image_for_cropping[ymin:ymax, xmin:xmax]
        #feature_vec = image_loader(crop_img)

        cv2.imwrite(crop_path+i[:-4]+'_'+str(idx)+i[-4:],crop_img)
        list_of_heads.append(i[:-4]+'_'+str(idx)+i[-4:])
        list_of_images.append(i)
        save_dir_list.append(crop_path)


    cv2.imwrite(save_dir+i[:-4]+'_eval_detected_v1'+i[-4:],image)

        # Clean up
    cv2.destroyAllWindows()



#


d:\users\micha\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
D:\object_detection\tf_model_dir\models\research\object_detection\utils\visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "d:\users\micha\appdata\local\programs\python\python35\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "d:\users\micha\appdata\local\programs\python\python35\lib\runpy.py", line 85, in _run_code
  

In [2]:
#

In [3]:
dat = pd.DataFrame({'head_name':list_of_heads,'original_image':list_of_images,'dir_loc':save_dir_list})
dat.to_csv('images_fgo_face/test_set_head_base_mapping.csv')